# Sequence generation for the pseudoRWM set of experiments

## Import statements and utilities

In [2]:
import numpy as np
import os
import matlab.engine
from alive_progress import alive_bar
import pandas as pd
from itertools import permutations

In [3]:
def shuffle_along_axis(arr, axis):
    idx = np.random.rand(*arr.shape).argsort(axis=axis)
    return np.take_along_axis(arr,idx,axis=axis)

def shuffled(arr):
    arr_shuffled = arr.copy()
    np.random.shuffle(arr_shuffled)
    return (arr_shuffled)

def choose_n_and_delete(arr, N):
    chosen = np.random.choice(arr, size=N, replace=False)
    arr = np.delete(arr, np.where(np.isin(arr, chosen)))
    return chosen, arr

In [4]:
def get_goals_with_rules(n_blocks_per_cond=2, n_colors=3, color_as_dim=False, diff_blocks=True):
    shapes = ["circle", "cross", "heart", "hexagon", "rhombus", "square", "star", "triangle"]
    colors = np.arange(1, 17).tolist() 
    sizes = ["large", "small"]
    fills = ["full", "empty"]
    features = ["shape", "size", "fill"]
    if color_as_dim:
        features = ["shape", "color", "fill"]
        n_colors = 2
        sizes = ["large"]
        colors = [2, 3, 5, 7, 9, 11, 12, 14]
    goals = {}
    nongoals = {}


    # Easy condition
    easy_rules = np.random.choice(features, size=n_blocks_per_cond, replace=False)

    for ri, rule in enumerate(easy_rules):
        these_shapes, shapes = choose_n_and_delete(shapes, 2)
        these_colors, colors = choose_n_and_delete(colors, n_colors)
        goals[f"easy_{ri}"] = []
        nongoals[f"easy_{ri}"] = []
        if rule == "shape":
            for color in these_colors:
                for size in sizes:
                    for fill in fills:
                        goals[f"easy_{ri}"].append(f"{size}_{fill}_{these_shapes[0]}_c{color}")
                        nongoals[f"easy_{ri}"].append(f"{size}_{fill}_{these_shapes[1]}_c{color}")
        elif rule == "size":
            for color in these_colors:
                for fill in fills:
                    for shape in these_shapes:
                        goals[f"easy_{ri}"].append(f"{sizes[0]}_{fill}_{shape}_c{color}")
                        nongoals[f"easy_{ri}"].append(f"{sizes[1]}_{fill}_{shape}_c{color}")
        elif rule == "fill":
            for color in these_colors:
                for size in sizes:
                    for shape in these_shapes:
                        goals[f"easy_{ri}"].append(f"{size}_{fills[0]}_{shape}_c{color}")
                        nongoals[f"easy_{ri}"].append(f"{size}_{fills[1]}_{shape}_c{color}")
        elif rule == "color":
            for size in sizes:
                for fill in fills:
                    for shape in these_shapes:
                        goals[f"easy_{ri}"].append(f"{size}_{fill}_{shape}_c{these_colors[0]}")
                        nongoals[f"easy_{ri}"].append(f"{size}_{fill}_{shape}_c{these_colors[1]}")

    # Hard condition  
    for ri in range(n_blocks_per_cond):
        these_shapes, shapes = choose_n_and_delete(shapes, 2)
        these_colors, colors = choose_n_and_delete(colors, n_colors)
        goals[f"hard_{ri}"] = []
        nongoals[f"hard_{ri}"] = []
        if not color_as_dim:
            for color in these_colors:
                goals[f"hard_{ri}"].append(f"{sizes[0]}_{fills[0]}_{these_shapes[0]}_c{color}")
                goals[f"hard_{ri}"].append(f"{sizes[0]}_{fills[1]}_{these_shapes[1]}_c{color}")
                goals[f"hard_{ri}"].append(f"{sizes[1]}_{fills[1]}_{these_shapes[0]}_c{color}")
                goals[f"hard_{ri}"].append(f"{sizes[1]}_{fills[0]}_{these_shapes[1]}_c{color}")
                nongoals[f"hard_{ri}"].append(f"{sizes[0]}_{fills[1]}_{these_shapes[0]}_c{color}")
                nongoals[f"hard_{ri}"].append(f"{sizes[0]}_{fills[0]}_{these_shapes[1]}_c{color}")
                nongoals[f"hard_{ri}"].append(f"{sizes[1]}_{fills[0]}_{these_shapes[0]}_c{color}")
                nongoals[f"hard_{ri}"].append(f"{sizes[1]}_{fills[1]}_{these_shapes[1]}_c{color}")
        else:   
            goals[f"hard_{ri}"].append(f"{sizes[0]}_{fills[0]}_{these_shapes[0]}_c{these_colors[0]}")
            goals[f"hard_{ri}"].append(f"{sizes[0]}_{fills[1]}_{these_shapes[1]}_c{these_colors[0]}")
            goals[f"hard_{ri}"].append(f"{sizes[0]}_{fills[1]}_{these_shapes[0]}_c{these_colors[1]}")
            goals[f"hard_{ri}"].append(f"{sizes[0]}_{fills[0]}_{these_shapes[1]}_c{these_colors[1]}")
            nongoals[f"hard_{ri}"].append(f"{sizes[0]}_{fills[1]}_{these_shapes[0]}_c{these_colors[0]}")
            nongoals[f"hard_{ri}"].append(f"{sizes[0]}_{fills[0]}_{these_shapes[1]}_c{these_colors[0]}")
            nongoals[f"hard_{ri}"].append(f"{sizes[0]}_{fills[0]}_{these_shapes[0]}_c{these_colors[1]}")
            nongoals[f"hard_{ri}"].append(f"{sizes[0]}_{fills[1]}_{these_shapes[1]}_c{these_colors[1]}")     

    if not diff_blocks:
        for bt in ["easy", "hard"]:
            goals[f"{bt}_1"] = goals[f"{bt}_0"]
            nongoals[f"{bt}_1"] = nongoals[f"{bt}_0"]

    return goals, nongoals


## Settings

In [5]:
exp_type = "pseudoRWMCatl4"  # choose from ["pseudoRWM", "pseudoRWMCtrl", "pseudoRWMConf", "pseudoRWMReps", "pseudoRWMConf2", "pseudoRWMReps2", "pseudoRWMConf3", "pseudoRWMReps3", "pseudoRWMNGoals", "pseudoRWMCatl", "pseudoRWMCatl2", "pseudoRWMCatl3", "pseudoRWMCatl4", "pseudoRWMCatl5"]
num_conditions = 10 # number of different file sequences to generate (change if needed)
reps_dict = {"pseudoRWM": 12, "pseudoRWMCtrl": 12, 
             "pseudoRWMConf": 11, "pseudoRWMReps": 11, 
             "pseudoRWMConf2": 11, "pseudoRWMReps2": 11, 
             "pseudoRWMConf3": 11, "pseudoRWMReps3": 11, 
             "pseudoRWMNGoals": 11, "pseudoRWMCatl": 11,
             "pseudoRWMCatl2": 11, "pseudoRWMCatl3":11, 
             "pseudoRWMCatl4": 11, "pseudoRWMCatl5": 11,}  # number of repetitions after the first presentation
num_reps = reps_dict[exp_type] # number of stimulus repetitions (after first presentation; change if needed)
exact_reps =  True if exp_type in ["pseudoRWMConf", "pseudoRWMReps", "pseudoRWMConf2", "pseudoRWMReps2", "pseudoRWMConf3", "pseudoRWMReps3", "pseudoRWMNGoals", "pseudoRWMCatl", "pseudoRWMCatl2", "pseudoRWMCatl3", "pseudoRWMCatl4", "pseudoRWMCatl5"] else False  # should the number of repetitions be exact or ok to exceed by 1?
use_matlab = False  # use matlab to call createstimsequence?

## Start MATLAB if needed

In [6]:
if use_matlab:
    m = matlab.engine.start_matlab()
    m.addpath(m.genpath("C:/Gaia/CCN Lab/Utilities/sequences/pseudoRWM/"), nargout=0)
else:
    from numpy.matlib import repmat

## Main sequence creation function

In [17]:
class pseudoRWMSequenceMaker:

    def __init__(self, exp_type, num_reps, use_matlab=True, num_conditions=10, exact_reps=False, 
                 max_consec_goal_reps=24):

        # ================ SETTINGS ==========================================================================
        assert exp_type in ["pseudoRWM", "pseudoRWMCtrl", "pseudoRWMConf", "pseudoRWMReps", "pseudoRWMConf2", "pseudoRWMReps2", "pseudoRWMConf3", "pseudoRWMReps3", "pseudoRWMNGoals", "pseudoRWMCatl", "pseudoRWMCatl2", "pseudoRWMCatl3", "pseudoRWMCatl4", "pseudoRWMCatl5"], f"{exp_type} is not a valid exp_type"
        self.exp_type = exp_type
        self.num_reps = num_reps
        self.num_conditions = num_conditions
        self.use_matlab = use_matlab
        self.exact_reps = exact_reps
        self.to_dir = f"C:/Gaia/CCN Lab/Utilities/sequences/pseudoRWM/{exp_type}/"
        self.has_group_row = self.exp_type in ["pseudoRWMConf", "pseudoRWMReps", "pseudoRWMConf2", "pseudoRWMReps2", "pseudoRWMConf3", "pseudoRWMReps3", "pseudoRWMNGoals", "pseudoRWMCatl", "pseudoRWMCatl2", "pseudoRWMCatl3", "pseudoRWMCatl4", "pseudoRWMCatl5"]

        self.num_keys = 3
        self.max_stims = 6
        self.block_structure = np.array([6, 6, 6, 6, 6, 6]) # 6 blocks: 3*type (Points vs Goals)
        self.num_blocks = len(self.block_structure)
        if exp_type in ["pseudoRWMNGoals", "pseudoRWMCatl", "pseudoRWMCatl2", "pseudoRWMCatl3", "pseudoRWMCatl4", "pseudoRWMCatl5"]:
            self.trial_type_structure = np.array([1, 0, 0, 1, 0, 0])  # 2 x (points, goals)
            if exp_type in ["pseudoRWMCatl", "pseudoRWMCatl2", "pseudoRWMCatl3", "pseudoRWMCatl4", "pseudoRWMCatl5"]:
                self.trial_group_structure = np.array([np.nan, 0, 1, np.nan, 2, 3])  # 2 x (points, easy, hard)
            # self.goal_block_type_structure = np.random.permutation(4)
        else:
            self.trial_type_structure = np.arange(self.num_blocks)%2  # half goals, half points
        self.num_goals_blocks = self.num_blocks - np.sum(self.trial_type_structure)
        self.num_goals_trials = np.sum(self.block_structure[np.where(self.trial_type_structure==0)])*(self.num_reps+1)
        self.max_consec_goal_reps = max_consec_goal_reps

        # R: columns represent keyboard keys
        # each column says how many stimuli will be associated with that key
        # e.g., 1, 2, 3 means 1 stimulus will be associated with key 0, 2, with key 1, 3 with key 2, 
        # for a total of 6 items
        if self.exp_type in ["pseudoRWM", "pseudoRWMCtrl", "pseudoRWMConf2", "pseudoRWMReps2", "pseudoRWMConf3", "pseudoRWMReps3", "pseudoRWMNGoals", "pseudoRWMCatl", "pseudoRWMCatl2", "pseudoRWMCatl3", "pseudoRWMCatl4", "pseudoRWMCatl5"]:
            self.R = np.vstack((
                np.array([1, 2, 3]),
                np.array([2, 2, 2]), 
                np.array([1, 2, 3]),
                np.array([2, 2, 2]), 
                np.array([1, 2, 3]),
                np.array([2, 2, 2])
            ))
        else:
            self.R = np.vstack((
                np.array([2, 2, 2]), 
                np.array([2, 2, 2]), 
                np.array([2, 2, 2]), 
                np.array([2, 2, 2]), 
                np.array([2, 2, 2]), 
                np.array([2, 2, 2])
            ))

        # goal images  
        self.all_fracts = np.arange(1, (self.num_goals_trials*2)+1)  # all fractal image numbers 
        self.all_ctrl_goal_sets = [[5, 17], [25, 26], [24, 30], [56, 59], [76, 77]]*2 # fractals to use in the Ctrl version includes goal and non-goal

    def _get_grouped_goal_sequences(self, all_images, trial_types, blocks, block_groups, block_seqprototypes, goal_block_type_structure=None, trial_group_structure=None):
        
        if self.exp_type == "pseudoRWMCatl":
            all_goal_images, all_nongoal_images = get_goals_with_rules()
        elif self.exp_type == "pseudoRWMCatl2":
            all_goal_images, all_nongoal_images = get_goals_with_rules(n_colors=1)
        elif self.exp_type in ["pseudoRWMCatl3", "pseudoRWMCatl4", "pseudoRWMCatl5"]:
            all_goal_images, all_nongoal_images = get_goals_with_rules(color_as_dim=True, diff_blocks=False)
        else:
            all_goal_images = all_images[0:len(all_images)//2] 
            all_nongoal_images = all_images[len(all_images)//2:]
        goal_images = np.array([])
        nongoal_images = np.array([])

        if self.exp_type == "pseudoRWMConf":
            n0, n1, n2 = 2, 2, 4   # for each block, we need 2 + 2 + 4 sets of goal/nongoal image pairs
        elif self.exp_type == "pseudoRWMReps":
            n0, n1, n2 = 6, 10, 8  # for each block, we need 6 + 10 + 8 sets of goal/nongoal image pairs
        elif self.exp_type == "pseudoRWMConf2":
            n0, n1, n2 = 1, 1, 2   # for each block, we need 1 + 1 + 2 sets of goal/nongoal image pairs
        elif self.exp_type == "pseudoRWMReps2":
            n0, n1, n2 = 6, 18, 12  # for each block, we need 6 + 18 + 12 sets of goal/nongoal image pairs
        elif self.exp_type == "pseudoRWMConf3":
            n0, n1, n2 = 1, 1, 2   # for each block, we need 1 + 1 + 2 sets of goal/nongoal image pairs
        elif self.exp_type == "pseudoRWMReps3":
            n0, n1, n2 = 6, 18, 12  # for each block, we need 6 + 18 + 12 sets of goal/nongoal image pairs
        elif self.exp_type == "pseudoRWMNGoals":
            n0, n1, n2, n3 = 1, 2, 3, 4  # in total, we need 1 + 2 + 3 + 4 sets of goal/nongoal image pairs
        elif self.exp_type == "pseudoRWMCatl":
            n0, n1, n2, n3 = 12, 12, 12, 12  # in total, we have 12 x 4 sets of goal/nongoal image pairs
        elif self.exp_type in ["pseudoRWMCatl2", "pseudoRWMCatl3", "pseudoRWMCatl4", "pseudoRWMCatl5"]:
            n0, n1, n2, n3 = 4, 4, 4, 4  # in total, we have 4 x 4 sets of goal/nongoal image pairs, each set repeated 3 times

        if self.exp_type in ["pseudoRWMCatl", "pseudoRWMCatl2", "pseudoRWMCatl3", "pseudoRWMCatl4", "pseudoRWMCatl5"]:
            group0_goal_images = all_goal_images["easy_0"]
            group1_goal_images = all_goal_images["hard_0"]
            group2_goal_images = all_goal_images["easy_1"]
            group3_goal_images = all_goal_images["hard_1"]
        elif self.exp_type == "pseudoRWMNGoals":
            group0_goal_images, all_goal_images = choose_n_and_delete(all_goal_images, (1, n0))
            group1_goal_images, all_goal_images = choose_n_and_delete(all_goal_images, (1, n1))
            group2_goal_images, all_goal_images = choose_n_and_delete(all_goal_images, (1, n2))
            group3_goal_images, all_goal_images = choose_n_and_delete(all_goal_images, (1, n3))
        else:
            group0_goal_images, all_goal_images = choose_n_and_delete(all_goal_images, (self.num_goals_blocks, n0))
            group1_goal_images, all_goal_images = choose_n_and_delete(all_goal_images, (self.num_goals_blocks, n1))
            group2_goal_images, all_goal_images = choose_n_and_delete(all_goal_images, (self.num_goals_blocks, n2))

        if self.exp_type in ["pseudoRWMCatl", "pseudoRWMCatl2", "pseudoRWMCatl3", "pseudoRWMCatl4", "pseudoRWMCatl5"]:
            group0_nongoal_images = all_nongoal_images["easy_0"]
            group1_nongoal_images = all_nongoal_images["hard_0"]
            group2_nongoal_images = all_nongoal_images["easy_1"]
            group3_nongoal_images = all_nongoal_images["hard_1"]
        elif self.exp_type == "pseudoRWMNGoals":
            group0_nongoal_images, all_nongoal_images = choose_n_and_delete(all_nongoal_images, (1, n0))
            group1_nongoal_images, all_nongoal_images = choose_n_and_delete(all_nongoal_images, (1, n1))
            group2_nongoal_images, all_nongoal_images = choose_n_and_delete(all_nongoal_images, (1, n2))            
            group3_nongoal_images, all_goal_images = choose_n_and_delete(all_goal_images, (1, n3))
        else:
            group0_nongoal_images, all_nongoal_images = choose_n_and_delete(all_nongoal_images, (self.num_goals_blocks, n0))
            group1_nongoal_images, all_nongoal_images = choose_n_and_delete(all_nongoal_images, (self.num_goals_blocks, n1))
            group2_nongoal_images, all_nongoal_images = choose_n_and_delete(all_nongoal_images, (self.num_goals_blocks, n2))


        if self.exp_type == "pseudoRWMReps2":
            shared_goal_images, all_goal_images = choose_n_and_delete(all_goal_images, (self.num_goals_blocks, 1))
            shared_nongoal_images, all_nongoal_images = choose_n_and_delete(all_nongoal_images, (self.num_goals_blocks, 1))
        elif self.exp_type == "pseudoRWMReps3":
            shared_goal_images, all_goal_images = choose_n_and_delete(all_goal_images, (self.num_goals_blocks, 3))
            shared_nongoal_images, all_nongoal_images = choose_n_and_delete(all_nongoal_images, (self.num_goals_blocks, 3))
        elif self.exp_type in ["pseudoRWMNGoals", "pseudoRWMCatl", "pseudoRWMCatl2", "pseudoRWMCatl3", "pseudoRWMCatl4", "pseudoRWMCatl5"]:
            all_groups_goal_images = [group0_goal_images, group1_goal_images, group2_goal_images, group3_goal_images]
            all_groups_nongoal_images = [group0_nongoal_images, group1_nongoal_images, group2_nongoal_images, group3_nongoal_images]
            all_groups_ns = [n0, n1, n2, n3]

        # block_goal_sequence_i is a dictionary for the current block where keys are stimuli IDs and values are sequences
        # of goal images for that stimulus
        for goal_block_i, block_i in enumerate(np.where(trial_types==0)[0]):
            ns = blocks[block_i]
            # dictionary for this block where keys are stimuli IDs and values are sequences of goal images for that stimulus
            block_goal_sequence_i = {}
            # dictionary for this block where keys are stimuli IDs and values are sequences of nongoal images for that stimulus
            block_nongoal_sequence_i = {}
            
            if self.exp_type == "pseudoRWMConf":
                # form goal and nongoal sequences for each group of the pseudoRWMConf condition
                group0_goal_sequences = np.repeat(group0_goal_images[goal_block_i], (self.num_reps+1)//2)
                group1_goal_sequences = np.hstack((
                    np.repeat(group1_goal_images[goal_block_i], (self.num_reps+1)//4), 
                    np.repeat(group1_nongoal_images[goal_block_i], (self.num_reps+1)//4)))
                group2_goal_sequences = np.repeat(group2_goal_images[goal_block_i], (self.num_reps+1)//4)
                
                group0_nongoal_sequences = np.repeat(group0_nongoal_images[goal_block_i], (self.num_reps+1)//2)
                group1_nongoal_sequences = np.hstack((
                    np.repeat(group1_nongoal_images[goal_block_i], (self.num_reps+1)//4), 
                    np.repeat(group1_goal_images[goal_block_i], (self.num_reps+1)//4)))
                group2_nongoal_sequences = np.repeat(group2_nongoal_images[goal_block_i], (self.num_reps+1)//4)

            elif self.exp_type == "pseudoRWMReps":
                # form goal and nongoal sequences for each group of the pseudoRWMReps condition
                one_fourth = (self.num_reps+1)//4  # 3 if there are 11(+1) reps
                one_sixth = (self.num_reps+1)//6   # 2 if there are 11(+1) reps
                
                group0_goal_sequences = np.hstack((
                    np.repeat(group0_goal_images[goal_block_i][0:one_fourth], one_fourth), group0_goal_images[goal_block_i][one_fourth:]))
                group1_goal_sequences = np.hstack((
                    np.repeat(group1_goal_images[goal_block_i][0], one_fourth), group1_goal_images[goal_block_i][1:]))
                group2_goal_sequences = np.hstack((
                    np.repeat(group2_goal_images[goal_block_i][0:one_sixth], one_fourth), group2_goal_images[goal_block_i][one_sixth:]))
                
                group0_nongoal_sequences = np.hstack((
                    np.repeat(group0_nongoal_images[goal_block_i][0:one_fourth], one_fourth), group0_nongoal_images[goal_block_i][one_fourth:]))
                group1_nongoal_sequences = np.hstack((
                    np.repeat(group1_nongoal_images[goal_block_i][0], one_fourth), group1_nongoal_images[goal_block_i][1:]))
                group2_nongoal_sequences = np.hstack((
                    np.repeat(group2_nongoal_images[goal_block_i][0:one_sixth], one_fourth), group2_nongoal_images[goal_block_i][one_sixth:]))
                
            elif self.exp_type == "pseudoRWMConf2":
                # form goal and nongoal sequences for each group of the pseudoRWMConf condition
                group0_goal_sequences = np.repeat(group0_goal_images[goal_block_i], (self.num_reps+1))     # just one goal image
                group1_goal_sequences = np.hstack((
                    np.repeat(group1_goal_images[goal_block_i], (self.num_reps+1)//2), 
                    np.repeat(group1_nongoal_images[goal_block_i], (self.num_reps+1)//2)))                 # one goal image, used as a goal half the time and as a nongoal the rest of the time
                group2_goal_sequences = np.repeat(group2_goal_images[goal_block_i], (self.num_reps+1)//2)  # two goal images
                
                group0_nongoal_sequences = np.repeat(group0_nongoal_images[goal_block_i], (self.num_reps+1))
                group1_nongoal_sequences = np.hstack((
                    np.repeat(group1_nongoal_images[goal_block_i], (self.num_reps+1)//2), 
                    np.repeat(group1_goal_images[goal_block_i], (self.num_reps+1)//2)))
                group2_nongoal_sequences = np.repeat(group2_nongoal_images[goal_block_i], (self.num_reps+1)//2)

            
            elif self.exp_type == "pseudoRWMReps2":
                # form goal and nongoal sequences for each group of the pseudoRWMReps condition
                # we want non-overlapping sequences, so they will end up being 2*(self.num_reps+1)
                # second_stim_order below needs to be edited accordingly
                three_quarters = 3*((self.num_reps+1)//4)  # 9 if there are 11(+1) reps
                half = (self.num_reps+1)//2                # 6 if there are 11(+1) reps
                one_quarter = (self.num_reps+1)//4         # 3 if there are 11(+1) reps
                
                group0_goal_sequences = np.hstack((
                    np.repeat(shared_goal_images[goal_block_i], three_quarters), 
                    group0_goal_images[goal_block_i][:len(group0_goal_images[goal_block_i])//2],
                    np.repeat(shared_goal_images[goal_block_i], three_quarters),
                    group0_goal_images[goal_block_i][len(group0_goal_images[goal_block_i])//2:]
                    ))
                group1_goal_sequences = np.hstack((
                    np.repeat(shared_goal_images[goal_block_i], one_quarter), 
                    group1_goal_images[goal_block_i][:len(group1_goal_images[goal_block_i])//2],
                    np.repeat(shared_goal_images[goal_block_i], one_quarter),
                    group1_goal_images[goal_block_i][len(group1_goal_images[goal_block_i])//2:]
                    ))
                group2_goal_sequences = np.hstack((
                    np.repeat(shared_goal_images[goal_block_i], half), 
                    group2_goal_images[goal_block_i][:len(group2_goal_images[goal_block_i])//2],
                    np.repeat(shared_goal_images[goal_block_i], half),
                    group2_goal_images[goal_block_i][len(group2_goal_images[goal_block_i])//2:]
                    ))
                
                group0_nongoal_sequences = np.hstack((
                    np.repeat(shared_nongoal_images[goal_block_i], three_quarters), 
                    group0_nongoal_images[goal_block_i][:len(group0_nongoal_images[goal_block_i])//2],
                    np.repeat(shared_nongoal_images[goal_block_i], three_quarters),
                    group0_nongoal_images[goal_block_i][len(group0_nongoal_images[goal_block_i])//2:]
                    ))
                group1_nongoal_sequences = np.hstack((
                    np.repeat(shared_nongoal_images[goal_block_i], one_quarter), 
                    group1_nongoal_images[goal_block_i][:len(group1_nongoal_images[goal_block_i])//2],
                    np.repeat(shared_nongoal_images[goal_block_i], one_quarter),
                    group1_nongoal_images[goal_block_i][len(group1_nongoal_images[goal_block_i])//2:]
                    ))
                group2_nongoal_sequences = np.hstack((
                    np.repeat(shared_nongoal_images[goal_block_i], half), 
                    group2_nongoal_images[goal_block_i][:len(group2_nongoal_images[goal_block_i])//2],
                    np.repeat(shared_nongoal_images[goal_block_i], half),
                    group2_nongoal_images[goal_block_i][len(group2_nongoal_images[goal_block_i])//2:]
                    ))
            
            elif self.exp_type == "pseudoRWMConf3":
                # form goal and nongoal sequences for each group of the pseudoRWMConf condition
                group0_goal_sequences = np.repeat(group0_goal_images[0], (self.num_reps+1))     # just one goal image, same one for all blocks
                group1_goal_sequences = np.hstack((
                    np.repeat(group1_goal_images[0], (self.num_reps+1)//2), 
                    np.repeat(group1_nongoal_images[0], (self.num_reps+1)//2)))                 # one goal image, used as a goal half the time and as a nongoal the rest of the time, same one for all blocks
                group2_goal_sequences = np.repeat(group2_goal_images[0], (self.num_reps+1)//2)  # two goal images, same two for all blocks
                
                group0_nongoal_sequences = np.repeat(group0_nongoal_images[0], (self.num_reps+1))
                group1_nongoal_sequences = np.hstack((
                    np.repeat(group1_nongoal_images[0], (self.num_reps+1)//2), 
                    np.repeat(group1_goal_images[0], (self.num_reps+1)//2)))
                group2_nongoal_sequences = np.repeat(group2_nongoal_images[0], (self.num_reps+1)//2)


            elif self.exp_type == "pseudoRWMReps3":
                # form goal and nongoal sequences for each group of the pseudoRWMReps condition
                # we want non-overlapping sequences, so they will end up being 2*(self.num_reps+1)
                # second_stim_order below needs to be edited accordingly
                three_quarters = 3*((self.num_reps+1)//4)  # 9 if there are 11(+1) reps
                half = (self.num_reps+1)//2                # 6 if there are 11(+1) reps
                one_quarter = (self.num_reps+1)//4         # 3 if there are 11(+1) reps
                
                group0_goal_sequences = np.hstack((
                    np.repeat(shared_goal_images[0][0], three_quarters),  # shared goal images are the same across blocks (first index of shared_goal_images is always 0), unique to each group (second index of shared_goal_images)
                    group0_goal_images[goal_block_i][:len(group0_goal_images[goal_block_i])//2],
                    np.repeat(shared_goal_images[0][0], three_quarters),
                    group0_goal_images[goal_block_i][len(group0_goal_images[goal_block_i])//2:]
                    ))
                group1_goal_sequences = np.hstack((
                    np.repeat(shared_goal_images[0][1], one_quarter), 
                    group1_goal_images[goal_block_i][:len(group1_goal_images[goal_block_i])//2],
                    np.repeat(shared_goal_images[0][1], one_quarter),
                    group1_goal_images[goal_block_i][len(group1_goal_images[goal_block_i])//2:]
                    ))
                group2_goal_sequences = np.hstack((
                    np.repeat(shared_goal_images[0][2], half), 
                    group2_goal_images[goal_block_i][:len(group2_goal_images[goal_block_i])//2],
                    np.repeat(shared_goal_images[0][2], half),
                    group2_goal_images[goal_block_i][len(group2_goal_images[goal_block_i])//2:]
                    ))
                
                group0_nongoal_sequences = np.hstack((
                    np.repeat(shared_nongoal_images[0][0], three_quarters), 
                    group0_nongoal_images[goal_block_i][:len(group0_nongoal_images[goal_block_i])//2],
                    np.repeat(shared_nongoal_images[0][0], three_quarters),
                    group0_nongoal_images[goal_block_i][len(group0_nongoal_images[goal_block_i])//2:]
                    ))
                group1_nongoal_sequences = np.hstack((
                    np.repeat(shared_nongoal_images[0][1], one_quarter), 
                    group1_nongoal_images[goal_block_i][:len(group1_nongoal_images[goal_block_i])//2],
                    np.repeat(shared_nongoal_images[0][1], one_quarter),
                    group1_nongoal_images[goal_block_i][len(group1_nongoal_images[goal_block_i])//2:]
                    ))
                group2_nongoal_sequences = np.hstack((
                    np.repeat(shared_nongoal_images[0][2], half), 
                    group2_nongoal_images[goal_block_i][:len(group2_nongoal_images[goal_block_i])//2],
                    np.repeat(shared_nongoal_images[0][2], half),
                    group2_nongoal_images[goal_block_i][len(group2_nongoal_images[goal_block_i])//2:]
                    ))
            
            elif self.exp_type == "pseudoRWMNGoals":
                # create a goal and a nongoal sequence for each block and stimulus
                for stim_n in range(ns):
                    # shuffle goal and nongoal images consistently for the same stimulus and separately for stimuli of the same group
                    stim_order = np.random.permutation(self.num_reps+1)
                    # goal sequences
                    block_goal_sequence_i[stim_n] = np.repeat(all_groups_goal_images[goal_block_type_structure[goal_block_i]], (self.num_reps+1)//(all_groups_ns[goal_block_type_structure[goal_block_i]]))[stim_order]
                    # nongoal_sequences
                    block_nongoal_sequence_i[stim_n] = np.repeat(all_groups_nongoal_images[goal_block_type_structure[goal_block_i]], (self.num_reps+1)//(all_groups_ns[goal_block_type_structure[goal_block_i]]))[stim_order]
                
                # create the full goal/nongoal sequences based on where stimuli are located
                tempgoalseq = np.empty((self.num_reps+1)*ns)
                tempnongoalseq = np.empty((self.num_reps+1)*ns)
                for stim_i in range(ns):
                    tempgoalseq[(block_seqprototypes[block_i]-1)==stim_i] = block_goal_sequence_i[stim_i]
                    tempnongoalseq[(block_seqprototypes[block_i]-1)==stim_i] = block_nongoal_sequence_i[stim_i]
                goal_images = np.hstack((goal_images, tempgoalseq))
                nongoal_images = np.hstack((nongoal_images, tempnongoalseq))

            elif self.exp_type in ["pseudoRWMCatl", "pseudoRWMCatl2", "pseudoRWMCatl3", "pseudoRWMCatl4", "pseudoRWMCatl5", "pseudoRWMCatl5"]:
                # create a goal and a nongoal sequence for each block and stimulus
                block_group_order = trial_group_structure.copy()
                block_group_order = block_group_order[~np.isnan(block_group_order)].astype(int)
                # shuffle goal and nongoal images consistently across the stimuli (Catl4)
                goal_fixed_order = np.random.permutation(4)
                nongoal_fixed_order = np.random.permutation(4)
                for stim_n in range(ns):
                    # shuffle goal and nongoal images consistently for the same stimulus and separately for stimuli of the same group (Catl/2/3)
                    stim_order = np.random.permutation(self.num_reps+1)
                    if self.exp_type in ["pseudoRWMCatl", "pseudoRWMCatl2", "pseudoRWMCatl3"]:
                        # goal sequences, shuffled
                        block_goal_sequence_i[stim_n] = shuffled(np.repeat(all_groups_goal_images[block_group_order[goal_block_i]], (self.num_reps+1)//(all_groups_ns[goal_block_type_structure[goal_block_i]]))[stim_order])
                        # nongoal_sequences, shuffled
                        block_nongoal_sequence_i[stim_n] = shuffled(np.repeat(all_groups_nongoal_images[block_group_order[goal_block_i]], (self.num_reps+1)//(all_groups_ns[goal_block_type_structure[goal_block_i]]))[stim_order])
                    elif self.exp_type in ["pseudoRWMCatl4"]:
                        # goal sequences, shuffled
                        block_goal_sequence_i[stim_n] = (np.repeat(np.array(all_groups_goal_images[block_group_order[goal_block_i]])[goal_fixed_order], (self.num_reps+1)//(all_groups_ns[goal_block_type_structure[goal_block_i]])))[stim_order]
                        # nongoal_sequences, shuffled in the same way
                        block_nongoal_sequence_i[stim_n] = (np.repeat(np.array(all_groups_nongoal_images[block_group_order[goal_block_i]])[nongoal_fixed_order], (self.num_reps+1)//(all_groups_ns[goal_block_type_structure[goal_block_i]])))[stim_order]
                    elif self.exp_type in ["pseudoRWMCatl5"]:
                        # goal sequences, shuffled but consistent with respect to categories
                        block_goal_sequence_i[stim_n] = (np.tile(all_groups_goal_images[block_group_order[goal_block_i]], (self.num_reps+1)//(all_groups_ns[goal_block_type_structure[goal_block_i]])))[stim_order]
                        # nongoal_sequences, shuffled in the same way
                        block_nongoal_sequence_i[stim_n] = (np.tile(all_groups_nongoal_images[block_group_order[goal_block_i]], (self.num_reps+1)//(all_groups_ns[goal_block_type_structure[goal_block_i]])))[stim_order]
                    
                # create the full goal/nongoal sequences based on where stimuli are located
                tempgoalseq = np.empty((self.num_reps+1)*ns, dtype=object)
                tempnongoalseq = np.empty((self.num_reps+1)*ns, dtype=object)
                for stim_i in range(ns):
                    tempgoalseq[(block_seqprototypes[block_i]-1)==stim_i] = block_goal_sequence_i[stim_i]
                    tempnongoalseq[(block_seqprototypes[block_i]-1)==stim_i] = block_nongoal_sequence_i[stim_i]
                goal_images = np.hstack((goal_images, tempgoalseq))
                nongoal_images = np.hstack((nongoal_images, tempnongoalseq))

            if self.exp_type not in ["pseudoRWMNGoals", "pseudoRWMCatl", "pseudoRWMCatl2", "pseudoRWMCatl3", "pseudoRWMCatl4", "pseudoRWMCatl5"]:   
                # shuffle goal and nongoal images consistently for the same stimulus and separately for stimuli of the same group
                first_stim_order = np.random.permutation(self.num_reps+1)
                second_stim_order = np.random.permutation(self.num_reps+1)
                if self.exp_type in ["pseudoRWMReps2", "pseudoRWMReps3"]:  # stimulus sequences should not be overlapping
                    second_stim_order += self.num_reps+1
                
                # get which stimulus is associated with which group 
                group_to_stim_dict = {}
                for key, value in block_groups[block_i].items():
                    if value not in group_to_stim_dict:
                        group_to_stim_dict[value] = []
                    group_to_stim_dict[value].append(key)
                # group_to_stim_dict[0][0] = the first stimulus of the first group, 
                # group_to_stim_dict[0][1] = the second stimulus of the first group, etc.
                # create a goal and a nongoal sequence for each block and stimulus
                for g_i, (group_goal_sequence, group_nongoal_sequence) in enumerate(zip(
                        [group0_goal_sequences, group1_goal_sequences, group2_goal_sequences], 
                        [group0_nongoal_sequences, group1_nongoal_sequences, group2_nongoal_sequences])):
                    for stim_n, stim_order in enumerate([first_stim_order, second_stim_order]):
                        # goal sequences
                        block_goal_sequence_i[group_to_stim_dict[g_i][stim_n]] = group_goal_sequence[stim_order]
                        # nongoal_sequences
                        block_nongoal_sequence_i[group_to_stim_dict[g_i][stim_n]] = group_nongoal_sequence[stim_order]
                
                # create the full goal/nongoal sequences based on where stimuli are located
                tempgoalseq = np.empty((self.num_reps+1)*ns)
                tempnongoalseq = np.empty((self.num_reps+1)*ns)
                for stim_i in range(ns):
                    tempgoalseq[(block_seqprototypes[block_i]-1)==stim_i] = block_goal_sequence_i[stim_i]
                    tempnongoalseq[(block_seqprototypes[block_i]-1)==stim_i] = block_nongoal_sequence_i[stim_i]
                goal_images = np.hstack((goal_images, tempgoalseq))
                nongoal_images = np.hstack((nongoal_images, tempnongoalseq))

        return goal_images, nongoal_images


    def make_sequences(self):
        with alive_bar(self.num_conditions, title=f"Making sequences", force_tty=True) as bar:

            all_goal_block_type_structures = [p for p in permutations([0, 1, 2, 3])]

            for s_i in range(self.num_conditions):

                goal_block_type_structure = all_goal_block_type_structures[s_i]

                # checks to be passed to approve a sequence
                consec_goal_reps_check = False

                while not consec_goal_reps_check:
                    # ================ BLOCKS ===========================================================================
                    # blocks: array of length n with each element representing the block's set size
                    blocks = self.block_structure.copy()

                    # ================ TRIAL TYPES =======================================================================
                    # trial_types: array  where each element represent the sequence
                    # of trial types (1 = Points, 0 = Goals) a participant will experience
                    trial_types = self.trial_type_structure.copy()
                    trial_group_structure = None
                    if self.exp_type == "pseudoRWMNGoals":
                        for i in range(0, self.num_blocks, self.num_blocks//2):
                            np.random.shuffle(trial_types[i:i+(self.num_blocks//2)]) 
                    elif self.exp_type in ["pseudoRWMCatl", "pseudoRWMCatl2", "pseudoRWMCatl3", "pseudoRWMCatl4", "pseudoRWMCatl5"]:
                        trial_group_structure = self.trial_group_structure.copy()
                        for i in range(0, self.num_blocks, self.num_blocks//2):
                            trial_type_order = np.random.permutation(self.num_blocks//2)
                            trial_types[i:i+(self.num_blocks//2)] = trial_types[trial_type_order+i]
                            trial_group_structure[i:i+(self.num_blocks//2)] = trial_group_structure[trial_type_order+i]
                    else:
                        for i in range(0, self.num_blocks, 2):
                            np.random.shuffle(trial_types[i:i+2]) 

                    # ================ RULES =============================================================================
                    # block_rules: a list where each element is a dictionary, with keys representing a stimulus image and
                    # values representing the solution for each stimulus
                    # here we shuffle within rows (only the first three columns), so that it's not always the same keys 
                    # that have 1, 2, or 3 stimuli associated with them (no difference for rows with 2, 2, 2) 
                    R_i = shuffle_along_axis(self.R, axis=1)  # mix up stim/action within the rule
                    block_rules = []
                    for block_i in range(self.num_blocks):
                        block_rules.append({i: k for i, k in enumerate([key_i for key_i in range(self.num_keys) for _ in range(R_i[block_i, key_i])])})

                    # ================ STIMULI ===========================================================================
                    # stim_sets: stimulus sets (folders from where images will be taken for each block)
                    stim_sets = (np.random.permutation(self.num_blocks)+1)
            
                    # block_stimuli: will contain dictionaries with an image number for each stimulus
                    block_stimuli = []
                    for block_i, ns in enumerate(blocks):
                        block_stimuli.append({i: s for i, s in enumerate((np.random.permutation(self.max_stims)+1)[0:ns])})

                    # block_seqprototypes: will contain dictionaries for each participant, with keys representing a set size and
                    # values as lists with a sequence of stimuli to be presented
                    # create a prototype (corresponding to stimuli rather than stimulus images) for each set size
                    block_seqprototypes = []
                    
                    # block_sequences: maps block_seqprototypes to corresponding stimulus sequences based on block_stimuli
                    block_sequences = []
                    
                    for block_i, ns in enumerate(blocks):
                        criterion_passed = False
                        if self.use_matlab:
                            while not criterion_passed:
                                temp_seqprototype = np.squeeze(np.array(m.createstimsequence(m.double(int(self.num_reps)), m.double(int(ns))))).astype(int)
                                # if we want the exact number of repetitions across stimuli, we need to pass this criterion before accepting the sequence
                                criterion_passed = np.all(np.unique(temp_seqprototype, return_counts=True)[1]==(self.num_reps+1)) or (not self.exact_reps)
                                # print(s_i, block_i, criterion_passed, np.unique(temp_seqprototype, return_counts=True)[1])
                            block_seqprototypes.append(temp_seqprototype)
                        else:
                            # worse (but faster) alternative if createstimsequence doesn't work
                            temp_seqprototype = []
                            for _ in range(self.num_reps+1):
                                temp_seqprototype = np.hstack((temp_seqprototype, (shuffled(np.arange(1, ns+1)))))
                            block_seqprototypes.append(temp_seqprototype)
                        # turn into stimuli (stimulus image number)
                        block_sequences.append(np.vectorize((block_stimuli[block_i]).get)(block_seqprototypes[block_i]-1))

                    # ================ GROUPS ===========================================================================
                    # only for some experiments, we divide stimuli into three groups that will receive different treatments 
                    # in terms of goal image presentation
                    # we need to ensure that stimuli in the same group have different correct keys
                    # because for these experiments each key is associated with two consecutive stimulus numbers, we can do
                    # this by making sure stimuli of the same group are associated with stimuli that are one number apart
                    # we also need to randomize which group is associated with the first, second, or third set of solutions
                    # block_groups: will contain dictionaries with a group number for each stimulus
                    block_groups = []
                    if exp_type in ["pseudoRWMCatl", "pseudoRWMCatl2", "pseudoRWMCatl3", "pseudoRWMCatl4", "pseudoRWMCatl5"]:
                        for block_i, ns in enumerate(blocks):
                            # same group within each block
                            block_groups.append({i: trial_group_structure[block_i] for i in range(ns)})
                    else:	
                        for block_i, ns in enumerate(blocks):
                            block_groups.append({i: g for i, g in enumerate(np.tile(np.random.permutation(ns//2), 2))})

                    # ================ GOAL IMAGES ======================================================================
                    all_images = shuffled(self.all_fracts)
                    if self.exp_type == "pseudoRWM":
                        goal_images = all_images[0:len(all_images)//2] 
                        nongoal_images = all_images[len(all_images)//2:]

                    elif self.exp_type == "pseudoRWMCtrl":
                        goal_set = self.all_ctrl_goal_sets[s_i]
                        # counterbalance which image is the goal and which one is the non-goal across participants
                        goal_images = np.repeat(goal_set[int(s_i+1 >= self.num_conditions//2)], self.num_goals_trials)
                        nongoal_images = np.repeat(goal_set[1-int(s_i+1 >= self.num_conditions//2)], self.num_goals_trials)

                    elif self.exp_type in ["pseudoRWMConf", "pseudoRWMReps", "pseudoRWMConf2", "pseudoRWMReps2", "pseudoRWMConf3", "pseudoRWMReps3", "pseudoRWMNGoals", "pseudoRWMCatl", "pseudoRWMCatl2", "pseudoRWMCatl3", "pseudoRWMCatl4", "pseudoRWMCatl5"]:
                        goal_images, nongoal_images = self._get_grouped_goal_sequences(all_images, trial_types, blocks, block_groups, block_seqprototypes, goal_block_type_structure, trial_group_structure)
                
                    # ================ CSV FILE ==========================================================================
                    # create csv
                    # rows: stim, correct key, set size, blocks, img_folders, img_nums, trial_type, goal_img, nongoal_img
                    goal_stim_count = 0
                    goal_block_count = 0

                    for block_i, ns in enumerate(blocks):
                        block_length = (self.num_reps + 1) * ns  # number of trials in a block

                        if self.exp_type in ["pseudoRWMCatl", "pseudoRWMCatl2", "pseudoRWMCatl3", "pseudoRWMCatl4", "pseudoRWMCatl5"]:
                            float_array_top = np.full((8, block_length), fill_value=np.nan, dtype=float)
                            float_array_bottom = np.full((1, block_length), fill_value=np.nan, dtype=float)
                            object_row = np.full((1, block_length), fill_value="", dtype=object)
                            this_block = np.vstack([float_array_top, object_row, float_array_bottom])
                        else:
                            this_block = np.full((9+self.has_group_row, block_length), np.nan)

                        _, unique_idx = np.unique(block_sequences[block_i], return_index=True)
                        block_cond = trial_types[block_i]
                        
                        this_block[0] = block_seqprototypes[block_i]  # stimulus number
                        this_block[1] = np.vectorize((block_rules[block_i]).get)(block_seqprototypes[block_i]-1) # correct key for the stimulus number
                        this_block[2] = np.repeat(ns, block_length)  # set size
                        this_block[3] = np.repeat(block_i+1, block_length)  # block number
                        this_block[4] = np.repeat(stim_sets[block_i], block_length)  # image folder
                        this_block[5] = block_sequences[block_i]  # stimulus number
                        this_block[6] = np.repeat(block_cond, block_length)  # trial type

                        if block_cond == 0:
                            this_block[7] = goal_images[goal_stim_count:goal_stim_count+block_length]
                            this_block[8] = nongoal_images[goal_stim_count:goal_stim_count+block_length]
                            goal_stim_count += block_length
                            goal_block_count += 1
                        
                        if self.has_group_row:
                            if self.exp_type == "pseudoRWMNGoals":
                                if block_cond == 0:
                                    this_block[9] = np.repeat(goal_block_type_structure[goal_block_count-1], (self.num_reps+1)*6)
                                else:
                                    this_block[9] = np.repeat(np.nan, (self.num_reps+1)*6)
                            else:
                                this_block[9] = np.vectorize((block_groups[block_i]).get)(block_seqprototypes[block_i]-1) # group for the stimulus ID

                        if block_i == 0:
                            train_seq = this_block
                            unique_stims = this_block[:, unique_idx]
                        else:
                            train_seq = np.column_stack((train_seq, this_block))
                            unique_stims = np.column_stack((unique_stims, this_block[:, unique_idx]))

                    # consec_goal_reps_check = True
                    colnames = ["stim", "correct_key", "set_size", "block", "img_folder", "stim_img", "trial_type", "goal_img", "nongoal_img"]
                    if self.has_group_row:
                        colnames += ["group"]
                    # print(np.sum(pd.DataFrame(train_seq.T, columns=colnames).query("trial_type == 0").goal_img.diff()==0))
                    if self.exp_type in ["pseudoRWMCatl", "pseudoRWMCatl2", "pseudoRWMCatl3", "pseudoRWMCatl4", "pseudoRWMCatl5"]:
                        consec_goal_reps_check = np.sum(pd.Series((pd.factorize(pd.DataFrame(train_seq.T, columns=colnames).query("trial_type == 0").goal_img)[0])).diff()==0) <= self.max_consec_goal_reps

                    else:
                        consec_goal_reps_check = np.sum(pd.DataFrame(train_seq.T, columns=colnames).query("trial_type == 0").goal_img.diff()==0) <= self.max_consec_goal_reps

                # save output
                # as a numpy array
                # np.savetxt(f"{self.to_dir}seq{s_i+1}_learning.csv", train_seq, delimiter=",")
                # np.savetxt(f"{self.to_dir}seq{s_i+1}_learning_test.csv", train_seq, delimiter=",")
                # as a pandas dataframe
                train_seq = pd.DataFrame(train_seq)
                train_seq.to_csv(f"{self.to_dir}seq{s_i+1}_learning.csv", index=False, header=False)
                bar()

        return 

## Create sequences

In [18]:
exp_type

'pseudoRWMCatl4'

In [19]:
# seqmkr = pseudoRWMSequenceMaker(exp_type=exp_type, num_reps=num_reps, num_conditions=num_conditions, use_matlab=use_matlab, exact_reps=exact_reps) 
seqmkr = pseudoRWMSequenceMaker(exp_type=exp_type, num_reps=num_reps, num_conditions=10, 
                                use_matlab=use_matlab, exact_reps=exact_reps, max_consec_goal_reps=250) 
# can do max_consec_goal_reps = 4 for Reps3, 16 for Conf3 (a bit slow), 12 for Catl, 48 for Catl2/3, 250 for Catl4, 300 for Catl5
seqmkr.make_sequences()

Making sequences |████████████████████████████████████████| 10/10 [100%] in 0.2s


## Checks

In [ ]:
df = pd.read_csv(f"{os.path.dirname(os.getcwd())}/{exp_type}/seq1_learning.csv", header=None).T

colnames = ["stim", "correct_key", "set_size", "block", "img_folder", "stim_img", "trial_type", "goal_img", "nongoal_img"]
if exp_type in ["pseudoRWMConf", "pseudoRWMReps", "pseudoRWMConf2", "pseudoRWMReps2", "pseudoRWMConf3", "pseudoRWMReps3", "pseudoRWMNGoals", "pseudoRWMCatl", "pseudoRWMCatl2", "pseudoRWMCatl3", "pseudoRWMCatl4", "pseudoRWMCatl5"]:
    colnames += ["group"]
df.columns = colnames
df["condition"] = np.where(df.goal_img.isna(), 1, 0)

In [40]:
df.head()

,stim,correct_key,set_size,block,img_folder,stim_img,trial_type,goal_img,nongoal_img,group,condition
0,6.0,2,6,1,6,5,0,large_full_hexagon_c14,large_empty_hexagon_c14,1.0,0
1,2.0,0,6,1,6,1,0,large_full_hexagon_c14,large_empty_hexagon_c14,1.0,0
2,3.0,1,6,1,6,3,0,large_full_hexagon_c14,large_empty_hexagon_c14,1.0,0
3,4.0,2,6,1,6,6,0,large_full_hexagon_c14,large_empty_hexagon_c14,1.0,0
4,1.0,0,6,1,6,2,0,large_full_hexagon_c14,large_empty_hexagon_c14,1.0,0


In [41]:
df.groupby(["block"]).correct_key.unique()

block
2.0    [0.0, 2.0, 1.0]
6.0    [1.0, 0.0, 2.0]
1            [2, 0, 1]
3            [2, 0, 1]
4            [0, 2, 1]
5            [1, 0, 2]
Name: correct_key, dtype: object

In [42]:
df.groupby(["block"]).group.unique()

block
2.0    [nan]
6.0    [nan]
1      [1.0]
3      [0.0]
4      [3.0]
5      [2.0]
Name: group, dtype: object

In [43]:
df.group.unique()

array(['1.0', nan, '0.0', '3.0', '2.0'], dtype=object)

In [50]:
for n in range(4):
    g = str(float(n))
    print(pd.Series(pd.factorize(df.query("group==@g & condition==0").goal_img)[0]).nunique())
#     assert df.query("group==@n").goal_img.nunique() == n+1

4
4
4
4


In [ ]:
for i in range(1, 11):
    # exp_type = "pseudoRWMCatl2"

    df = pd.read_csv(f"{os.path.dirname(os.getcwd())}/{exp_type}/seq{i}_learning.csv", header=None).T

    colnames = ["stim", "correct_key", "set_size", "block", "img_folder", "stim_img", "trial_type", "goal_img", "nongoal_img"]
    if exp_type in ["pseudoRWMConf", "pseudoRWMReps", "pseudoRWMConf2", "pseudoRWMReps2", "pseudoRWMConf3", "pseudoRWMReps3", "pseudoRWMNGoals", "pseudoRWMCatl", "pseudoRWMCatl2", "pseudoRWMCatl3", "pseudoRWMCatl4", "pseudoRWMCatl5"]:
        colnames += ["group"]
    df.columns = colnames
    df["condition"] = np.where(df.goal_img.isna(), 1, 0)

    goal_img_by_group = df.query("condition==0").groupby("group").goal_img.unique()
    display(pd.DataFrame(goal_img_by_group.tolist(), index=goal_img_by_group.index))

    nongoal_img_by_group = df.query("condition==0").groupby("group").nongoal_img.unique()
    display(pd.DataFrame(nongoal_img_by_group.tolist(), index=nongoal_img_by_group.index))

    display(df.query("condition==0").groupby("block").group.first())

,0,1,2,3
group,,,,
0.0,large_full_rhombus_c5,large_full_heart_c5,large_full_heart_c9,large_full_rhombus_c9
1.0,large_full_hexagon_c14,large_empty_circle_c14,large_empty_hexagon_c12,large_full_circle_c12
2.0,large_full_rhombus_c5,large_full_heart_c9,large_full_heart_c5,large_full_rhombus_c9
3.0,large_full_hexagon_c14,large_full_circle_c12,large_empty_circle_c14,large_empty_hexagon_c12


,0,1,2,3
group,,,,
0.0,large_empty_rhombus_c5,large_empty_heart_c5,large_empty_heart_c9,large_empty_rhombus_c9
1.0,large_empty_hexagon_c14,large_full_circle_c14,large_full_hexagon_c12,large_empty_circle_c12
2.0,large_empty_rhombus_c5,large_empty_heart_c9,large_empty_heart_c5,large_empty_rhombus_c9
3.0,large_empty_hexagon_c14,large_empty_circle_c12,large_full_circle_c14,large_full_hexagon_c12


block
1    1.0
3    0.0
4    3.0
5    2.0
Name: group, dtype: object

,0,1,2,3
group,,,,
0.0,large_full_star_c12,large_empty_circle_c12,large_empty_star_c12,large_full_circle_c12
1.0,large_empty_triangle_c9,large_empty_square_c14,large_full_square_c9,large_full_triangle_c14
2.0,large_empty_star_c12,large_full_circle_c12,large_full_star_c12,large_empty_circle_c12
3.0,large_full_triangle_c14,large_full_square_c9,large_empty_square_c14,large_empty_triangle_c9


,0,1,2,3
group,,,,
0.0,large_full_star_c3,large_empty_circle_c3,large_empty_star_c3,large_full_circle_c3
1.0,large_full_triangle_c9,large_full_square_c14,large_empty_square_c9,large_empty_triangle_c14
2.0,large_empty_star_c3,large_full_circle_c3,large_full_star_c3,large_empty_circle_c3
3.0,large_empty_triangle_c14,large_empty_square_c9,large_full_square_c14,large_full_triangle_c9


...

,0,1,2,3
group,,,,
0.0,large_empty_hexagon_c5,large_empty_cross_c2,large_empty_hexagon_c2,large_empty_cross_c5
1.0,large_full_triangle_c12,large_empty_triangle_c3,large_full_rhombus_c3,large_empty_rhombus_c12
2.0,large_empty_hexagon_c2,large_empty_cross_c5,large_empty_hexagon_c5,large_empty_cross_c2
3.0,large_full_triangle_c12,large_empty_triangle_c3,large_empty_rhombus_c12,large_full_rhombus_c3


block
1    0.0
3    1.0
4    3.0
5    2.0
Name: group, dtype: object

,0,1,2,3
group,,,,
0.0,large_full_heart_c7,large_empty_hexagon_c7,large_full_hexagon_c7,large_empty_heart_c7
1.0,large_empty_cross_c5,large_full_triangle_c5,large_full_cross_c12,large_empty_triangle_c12
2.0,large_full_heart_c7,large_empty_heart_c7,large_full_hexagon_c7,large_empty_hexagon_c7
3.0,large_full_triangle_c5,large_empty_triangle_c12,large_full_cross_c12,large_empty_cross_c5


,0,1,2,3
group,,,,
0.0,large_full_heart_c3,large_empty_hexagon_c3,large_full_hexagon_c3,large_empty_heart_c3
1.0,large_full_cross_c5,large_empty_triangle_c5,large_empty_cross_c12,large_full_triangle_c12
2.0,large_full_heart_c3,large_empty_heart_c3,large_full_hexagon_c3,large_empty_hexagon_c3
3.0,large_empty_triangle_c5,large_full_triangle_c12,large_empty_cross_c12,large_full_cross_c5


block
1    1.0
3    0.0
5    2.0
6    3.0
Name: group, dtype: object

In [53]:
df.loc[df.condition==0, "both_imgs"] = df.loc[df.condition==0, "goal_img"]+"_"+df.loc[df.condition==0, "nongoal_img"]
df.query("condition==0").groupby("stim").both_imgs.unique().agg(list)

stim
1.0    [large_empty_hexagon_c3_large_full_hexagon_c3,...
2.0    [large_empty_hexagon_c3_large_full_hexagon_c3,...
3.0    [large_empty_triangle_c7_large_full_triangle_c...
4.0    [large_empty_triangle_c7_large_full_triangle_c...
5.0    [large_empty_hexagon_c3_large_full_hexagon_c3,...
6.0    [large_full_triangle_c3_large_empty_triangle_c...
Name: both_imgs, dtype: object

In [ ]:
goal_img_by_group = df.query("condition==0").groupby("group").goal_img.unique()
pd.DataFrame(goal_img_by_group.tolist(), index=goal_img_by_group.index)

,0,1,2,3
group,,,,
0.0,large_full_square_c14,large_full_star_c14,large_full_star_c2,large_full_square_c2
1.0,large_full_triangle_c5,large_empty_heart_c5,large_full_heart_c12,large_empty_triangle_c12
2.0,large_full_square_c14,large_full_star_c14,large_full_square_c2,large_full_star_c2
3.0,large_empty_triangle_c12,large_full_triangle_c5,large_empty_heart_c5,large_full_heart_c12


In [29]:
nongoal_img_by_group = df.query("condition==0").groupby("group").nongoal_img.unique()
pd.DataFrame(nongoal_img_by_group.tolist(), index=nongoal_img_by_group.index)

,0,1,2,3
group,,,,
0.0,large_empty_square_c14,large_empty_star_c14,large_empty_square_c2,large_empty_star_c2
1.0,large_empty_heart_c12,large_full_triangle_c12,large_empty_triangle_c5,large_full_heart_c5
2.0,large_empty_square_c14,large_empty_square_c2,large_empty_star_c2,large_empty_star_c14
3.0,large_empty_triangle_c5,large_full_triangle_c12,large_full_heart_c5,large_empty_heart_c12


In [13]:
df.query("condition==0").groupby("block").group.first()

block
1    0.0
3    1.0
5    2.0
6    3.0
Name: group, dtype: object

In [12]:
goal_img_by_block = df.query("condition==0").groupby("block").goal_img.unique()
pd.DataFrame(goal_img_by_block.tolist(), index=goal_img_by_block.index)

,0,1,2,3,4,5,6,7,8,9,10,11
block,,,,,,,,,,,,
1,large_empty_triangle_c3,large_empty_triangle_c14,large_full_triangle_c14,large_full_hexagon_c14,large_full_triangle_c15,large_empty_hexagon_c14,large_full_hexagon_c3,large_empty_hexagon_c15,large_empty_hexagon_c3,large_full_hexagon_c15,large_full_triangle_c3,large_empty_triangle_c15
3,large_empty_circle_c9,small_full_circle_c10,large_full_heart_c16,small_empty_heart_c9,small_full_circle_c9,small_empty_heart_c10,small_full_circle_c16,large_empty_circle_c10,large_full_heart_c9,large_empty_circle_c16,small_empty_heart_c16,large_full_heart_c10
5,small_empty_rhombus_c5,small_empty_rhombus_c7,large_empty_rhombus_c5,small_full_rhombus_c7,large_full_rhombus_c4,large_full_rhombus_c5,large_empty_rhombus_c7,small_full_rhombus_c4,small_empty_rhombus_c4,large_full_rhombus_c7,large_empty_rhombus_c4,small_full_rhombus_c5
6,large_full_cross_c11,small_full_star_c6,small_empty_cross_c11,large_empty_star_c6,large_full_cross_c1,small_empty_cross_c1,small_full_star_c11,large_empty_star_c11,small_empty_cross_c6,large_empty_star_c1,large_full_cross_c6,small_full_star_c1


In [61]:
np.any(np.isin(df.query("condition==0").goal_img.unique(), df.query("condition==0").nongoal_img.unique()))

False

In [37]:
df = None
for c in range(seqmkr.num_conditions):
    new_df = pd.read_csv(f"{os.path.dirname(os.getcwd())}/{exp_type}/seq{c+1}_learning.csv", header=None).T
    colnames = ["stim", "correct_key", "set_size", "block", "img_folder", "stim_img", "trial_type", "goal_img", "nongoal_img"]+["group"]
    new_df.columns = colnames 
    new_df["condition"] = np.where(new_df.goal_img.isna(), 1, 0)
    # print(df.group.dropna().unique())
    new_df["group_cat"] = pd.Categorical(new_df.group.fillna(-1), categories=[-1, 0, 1, 2, 3])
    new_df["c"] = c
    if df is None:
        df = new_df.copy()
    else:
        df = pd.concat((df, new_df))
print(f"{df.groupby('block').group_cat.value_counts(normalize=True)}")

block  group_cat
1.0    -1           1.0
       0            0.0
       1            0.0
       2            0.0
       3            0.0
2.0    -1           1.0
       0            0.0
       1            0.0
       2            0.0
       3            0.0
3.0    -1           1.0
       0            0.0
       1            0.0
       2            0.0
       3            0.0
4.0    -1           1.0
       0            0.0
       1            0.0
       2            0.0
       3            0.0
1      -1           NaN
       0            NaN
       1            NaN
       2            NaN
       3            NaN
2      -1           NaN
       0            NaN
       1            NaN
       2            NaN
       3            NaN
3      -1           NaN
       0            NaN
       1            NaN
       2            NaN
       3            NaN
5      -1           NaN
       0            NaN
       1            NaN
       2            NaN
       3            NaN
6      -1           NaN

In [39]:
for i in range(1,11):
    df = pd.read_csv(f"{os.path.dirname(os.getcwd())}/{exp_type}/seq{i}_learning.csv", header=None).T

    colnames = ["stim", "correct_key", "set_size", "block", "img_folder", "stim_img", "trial_type", "goal_img", "nongoal_img"]
    if seqmkr.has_group_row:
        colnames += ["group"]
    df.columns = colnames
    df["condition"] = np.where(df.goal_img.isna(), 1, 0)
    print(len(df.img_folder.astype(int).unique()), df.img_folder.astype(int).unique())

6 [1 5 6 2 4 3]
6 [1 2 3 4 5 6]
6 [4 2 6 1 5 3]
6 [6 3 4 1 2 5]
6 [1 6 4 2 5 3]
6 [3 1 5 6 4 2]
6 [3 1 5 6 4 2]
6 [6 3 4 5 1 2]
6 [4 3 5 2 6 1]
6 [4 6 2 1 5 3]


In [41]:
for n in range(1, 11):
    df = pd.read_csv(f"{os.path.dirname(os.getcwd())}/{exp_type}/seq{n}_learning.csv", header=None).T

    colnames = ["stim", "correct_key", "set_size", "block", "img_folder", "stim_img", "trial_type", "goal_img", "nongoal_img"]
    if seqmkr.has_group_row:
        colnames += ["group"]
    df.columns = colnames
    df["condition"] = np.where(df.goal_img.isna(), 1, 0)

    temp_df = df.groupby(["block", "group", "stim"], as_index=False).correct_key.first()
    temp_df.rename(columns={"correct_key": "stim_corr_key"}, inplace=True)
    temp_df["other_stim_corr_key"] = temp_df.loc[np.tile([0, -2], int(np.ceil(len(temp_df)/2))) + np.arange(1, len(temp_df)+1), "stim_corr_key"].reset_index(drop=True)       
    print(n, temp_df.query("stim_corr_key == other_stim_corr_key").block.astype(int).unique())

1 [1 2 5 6]
2 [1 2 5 6]
3 [1 2 5 6]
4 [1 3 5 6]
5 [1 2 5 6]
6 [1 2 5 6]
7 [2 3 5 6]
8 [2 3 5 6]
9 [2 3 5 6]
10 [2 3 5 6]


In [45]:
for n in range(1, 11):
    df = pd.read_csv(f"{os.path.dirname(os.getcwd())}/{exp_type}/seq{n}_learning.csv", header=None).T

    colnames = ["stim", "correct_key", "set_size", "block", "img_folder", "stim_img", "trial_type", "goal_img", "nongoal_img"]
    if seqmkr.has_group_row:
        colnames += ["group"]
    df.columns = colnames
    df["condition"] = np.where(df.goal_img.isna(), 1, 0)

    # print(n, np.sum(df.query("condition == 0").goal_img.diff().fillna(999) == 0))
    print(n, np.sum((pd.Series(pd.factorize(df.query("condition == 0").goal_img)[0])).diff().fillna(999) == 0))

1 10
2 10
3 8
4 10
5 10
6 10
7 10
8 10
9 8
10 9


In [46]:
len(df.query("condition == 0"))

288

In [47]:
temp_df = df.groupby(["block", "group", "stim"], as_index=False).correct_key.first()
temp_df.rename(columns={"correct_key": "stim_corr_key"}, inplace=True)
temp_df["other_stim_corr_key"] = temp_df.loc[np.tile([0, -2], int(np.ceil(len(temp_df)/2))) + np.arange(1, len(temp_df)+1), "stim_corr_key"].reset_index(drop=True)       
temp_df.query("stim_corr_key == other_stim_corr_key")  

,block,group,stim,stim_corr_key,other_stim_corr_key
0,2,1.0,1.0,0,0
1,2,1.0,2.0,0,0
2,2,1.0,3.0,1,1
3,2,1.0,4.0,1,1
4,2,1.0,5.0,2,2
5,2,1.0,6.0,2,2
6,3,0.0,1.0,0,0
7,3,0.0,2.0,0,0
8,3,0.0,3.0,1,1
9,3,0.0,4.0,1,1


In [48]:
temp_df = df.query("condition == 0").groupby(["block", "group", "stim"]).correct_key.value_counts()
temp_df

block  group  stim  correct_key
2      1.0    1.0   0              12
              2.0   0              12
              3.0   1              12
              4.0   1              12
              5.0   2              12
              6.0   2              12
3      0.0    1.0   0              12
              2.0   0              12
              3.0   1              12
              4.0   1              12
              5.0   1              12
              6.0   2              12
5      2.0    1.0   0              12
              2.0   0              12
              3.0   0              12
              4.0   1              12
              5.0   1              12
              6.0   2              12
6      3.0    1.0   0              12
              2.0   0              12
              3.0   1              12
              4.0   1              12
              5.0   2              12
              6.0   2              12
Name: correct_key, dtype: int64

In [ ]:
df.groupby("block").trial_type.value_counts()

In [ ]:
df.query("condition==0").groupby(["block", "group", "stim"]).goal_img.apply(lambda x: np.sort(pd.unique(x)))

In [ ]:
df.groupby(["block", "group"]).goal_img.apply(lambda x: np.sort(pd.unique(x)))

In [ ]:
df.groupby(["block", "group"]).goal_img.unique()

In [ ]:
df.groupby(["block", "group"]).goal_img.nunique()

In [ ]:
df.groupby(["block", "group"]).nongoal_img.apply(lambda x: np.sort(pd.unique(x)))

In [ ]:
df.groupby(["block", "group"]).nongoal_img.unique()

In [ ]:
df.groupby(["block", "group"]).goal_img.apply(lambda x: np.sort(np.unique(x, return_counts=True)[1]))

In [ ]:
df.groupby(["block", "group", "stim"]).goal_img.apply(lambda x: np.sort(np.unique(x, return_counts=True)[1]))

In [ ]:
df.groupby(["block", "stim"]).goal_img.apply(lambda x: np.sort(np.unique(x, return_counts=True)[1]))

In [ ]:
df.groupby(["block", "group", "stim"]).correct_key.first()

In [ ]:
df.groupby(["block", "group"]).goal_img.apply(lambda x: np.sort(np.unique(x, return_counts=True)[1]))

In [ ]:
df.groupby(["block", "group"]).goal_img.apply(lambda x: len(np.unique(x)))

In [ ]:
df.groupby(["block", "goal_img"]).stim.apply(lambda x: len(np.unique(x)))

In [78]:
# df.to_csv("test_seq.csv")

## Quit MATLAB if needed

In [79]:
# quit matlab
if use_matlab:
    m.exit()